In [ ]:
import pandas as pd


In [ ]:
csv_path = "../00_data/UKB_demographics_covariates.csv"
merged_df = pd.read_csv(csv_path)
merged_df["ID"] = merged_df["ID"].astype(str)

In [ ]:
merged_df.to_csv("../00_data/UKB_demographics_covariates_subset_healthy.csv", index=False)
merged_df.rename(columns={"Age": "label_Age"}, inplace=True)
merged_df.to_csv("../00_data/age_label/all_ages_healthy.csv", index=False)

In [ ]:
#MERGE LABEL COL CLINICAL HERE TO DROP

In [ ]:
for col in merged_df.columns:
    print(col)

In [ ]:

# Filter the dataframe
merged_df = merged_df[merged_df["a_diag_epilep"] == 0]
merged_df = merged_df[merged_df["a_diag_parkinson"] == 0]
filtered_df = merged_df[
    (merged_df["a_emo_phq9_kat"] != 7777) &
    (merged_df["a_emo_gad7_dia"] != 9999) &
    (merged_df["a_alk_rein"] > 0) &
    (merged_df["a_alk_konsum"] != -99) &
    (merged_df["a_alk_audit_c"] != -99) &
    (merged_df["a_smok_stat_noint"].isin([1, 2])) &
    (merged_df["a_anthro_bmi"] != 7777)
    
]

all_diagnoses = [
    ("depression_and_angst", 2), ("depression_and_angst", 3), ("depression_and_angst", 4),
    ("depression", 2), ("depression", 3), ("depression", 4),
    ("angst", 2), ("angst", 3), ("angst", 4),
    ("alkohol", 70), ("alkohol", 80), ("alkohol", 90),
    ("alkohol_audit", 9), ("alkohol_audit", 10)  # ,  ("alkohol_audit", 11)
]

for diagnosis_crit, thresh in all_diagnoses:
    if diagnosis_crit == "depression_and_angst":
        filtered_df[f'diagnosis_{diagnosis_crit}_{thresh}'] = filtered_df.apply(
            lambda row: 1 if (row['a_emo_phq9_kat'] >= thresh) or \
                             (row['a_emo_gad7_dia'] >= thresh) else 0,
            axis=1
        )
    elif diagnosis_crit == "depression":
        filtered_df[f'diagnosis_{diagnosis_crit}_{thresh}'] = filtered_df.apply(
            lambda row: 1 if row['a_emo_phq9_kat'] >= thresh else 0,
            axis=1
        )
    elif diagnosis_crit == "angst":
        filtered_df[f'diagnosis_{diagnosis_crit}_{thresh}'] = filtered_df.apply(
            lambda row: 1 if row['a_emo_gad7_dia'] >= thresh else 0,
            axis=1
        )
    elif diagnosis_crit == "alkohol":
        filtered_df[f'diagnosis_{diagnosis_crit}_{thresh}'] = filtered_df.apply(
            lambda row: 1 if row['a_alk_rein'] >= thresh and row['a_alk_konsum'] == 2 else 0,
            axis=1
        )
    elif diagnosis_crit == "alkohol_audit":
        filtered_df[f'diagnosis_{diagnosis_crit}_{thresh}'] = filtered_df.apply(
            lambda row: 1 if row['a_alk_audit_c'] >= thresh else 0,
            axis=1
        )
        

In [ ]:
#wen all diagnose are zero set health col to 1
filtered_df["healthy"] = filtered_df.apply(
    lambda row: 1 if sum([row[f'diagnosis_{diagnosis_crit}_{thresh}'] for diagnosis_crit, thresh in all_diagnoses]) == 0 else 0,
    axis=1
)

In [ ]:
Healthy_people = filtered_df[filtered_df["healthy"] == 1]

In [ ]:
age_df = pd.read_csv("/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/00_NAKO/00_data/age_label/all_ages_all_ids.csv")
if "Unnamed: 0" in age_df.columns:
    age_df.drop(columns=["Unnamed: 0"], inplace=True)
len(age_df)

In [ ]:
#merge the ID for healthy people
age_df["ID"] = age_df["ID"].astype(str)
merged_df = pd.merge(age_df, Healthy_people, on="ID", how="inner")
final_all_ages = merged_df[age_df.columns]

In [ ]:
len(final_all_ages)

In [ ]:
final_all_ages.rename(columns={"Age": "label_Age"}, inplace=True)

In [ ]:
#save the merged df as all ages.csv
final_all_ages.to_csv("/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/00_NAKO/00_data/age_label/all_ages_healthy.csv", index=False)

In [ ]:
demographics = pd.read_csv("/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/00_NAKO/00_data/NAKO_demographics_covariates.csv")
len(demographics)

In [ ]:
demographics

In [ ]:
#merges with Healtey people
demographics["ID"] = demographics["ID"].astype(str)
merged_df = pd.merge(demographics, Healthy_people, on="ID", how="inner")
final_demographics = merged_df[demographics.columns]

In [ ]:
#Dorp outlier for BMI a_anthro_bmi
z_scores = (merged_df['a_anthro_bmi'] - merged_df['a_anthro_bmi'].mean()) / merged_df['a_anthro_bmi'].std()
threshold = 3
df_bmi = merged_df[abs(z_scores) <= threshold]
df_bmi = df_bmi[["ID","a_anthro_bmi"]]

final_demographics = pd.merge(final_demographics, df_bmi, on="ID", how="inner")

In [ ]:
final_demographics.rename(columns={"a_anthro_bmi": "BMI"}, inplace=True)

In [ ]:
len(final_demographics)
final_demographics.to_csv("/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/00_NAKO/00_data/NAKO_demographics_covariates_subset_healthy.csv", index=False)